In [ ]:
### Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import networkx as nx
import heapq
import time
from sklearn.cluster import KMeans
from utils_discrete import GraphConstructionDiscretization, biobjective_search, extract_costs, check_pareto_optimality, plot_costs, reconstruct_solution_paths, plot_map_with_path, sample_representative_paths

In [ ]:
# =============================================================================
# Constants, UAV parameters, and initial conditions
# =============================================================================
# Define QZ circles as tuples (x, y, radius, only_electric_radius, risk_limit, toggle_only_electric_or_no_path) toggle_only_electric_or_no_path = 0 for no path, 1 for only electric

# Map with two QZ circles
map_qz = [(0.0, 0.0, 6.0, 2.0, 30, 1), (12.0, 10.0, 4.0, 1.5, 30, 1)]
# max_risk_limit = 2 / 3 * sum([circle[-2] for circle in map_qz]) ### Total risk limit i.e. limit on the sum of all the risk limits
# acceptable_risk_limit = 1 / 3 * sum([circle[-2] for circle in map_qz])
start = (-5, -5)       # Starting point
goal = (15, 15)        # Goal point

# UAV characteristics
alpha = 10                      ### Discharge rate 
recharge_factor = 2             ### Factor by which recharge rate less than discharge rate
beta = alpha / recharge_factor  ### Recharge rate

q_min, q_max, q_act = 20, 100, 70 ### Minimum SOC limit, maximum SOC limit, and SOC at the state
discretization_angle = 10         ### Discretization angle for QZ in degrees

In [ ]:
# =============================================================================
# Graph Construction and Node Creation
# =============================================================================
# Instantiate the graph construction object.
graph_object = GraphConstructionDiscretization(map_qz, start, goal, q_min, q_max, q_act, alpha, beta, discretization_angle)

start_time = time.time()

# Create nodes ( x_pos, y_pos, qz_index, SOC) for the graph 
# Create the index maps and reverse index maps mapping nodes to node index. 
nodes, index_map, reverse_index_map = graph_object.create_nodes()

# Build the visibility graph
graph_object.build_visibility_graph(reverse_index_map)
end_time = time.time()

print(f"The time req for graph construction: {end_time-start_time}")

## Assign heuristic cost to each node
##
graph_object.assign_heuristic_costs(reverse_index_map)

# Run the biobjective search
start_state = "s"
goal_state = "g"

start_time = time.time()
sols, g2_min = biobjective_search(graph_object, start_state, goal_state, reduce_factor=1)
end_time = time.time()
print(f"The time req for biobjective_search: {end_time-start_time}")

In [ ]:

# =============================================================================
# Pareto Optimality Check and Plotting
# =============================================================================

# Extract cost values for the goal state
fuel_costs, risk_costs = extract_costs(sols, goal_state)

### Check Pareto optimality
### For solution to be Pareto optimal, it must be non-dominated, therefore, the list of dominated indices shoulb be epty
dominated_indices = check_pareto_optimality(fuel_costs, risk_costs)
print("Dominated solution indices:", dominated_indices)

# Plot the results
plot_costs(fuel_costs, risk_costs)

# Reconstruct and print all solution paths
solution_paths = reconstruct_solution_paths(sols, start_state, goal_state)
print("Extracted Solution Paths:")
for spath in solution_paths:
    print(spath)

In [ ]:
# =============================================================================
# Plotting the paths
# =============================================================================

### Select a single path for plotting, the tuple after "g" represents the fuel cost and the risk cost respectively
### Need to select one the path from the list of solution paths
single_path = ['s', 22, 11, 53, 48, 'g', (20.3207009673128, 28.0246597023148)]

### Selection of multiple paths for plotting evenly based on fuel cost
### Sort the paths based on the fuel cost
sorted_paths = sorted(solution_paths, key=lambda x: x[-1][0])
sampled_paths = sample_representative_paths(sorted_paths, num_path=5)

### Plot a single selected path
plot_map_with_path(graph_object, map_qz, start, goal, [single_path], reverse_index_map)

### Plot multiple sampled paths
plot_map_with_path(graph_object, map_qz, start, goal, sampled_paths, reverse_index_map)

# =============================================================================
# End of Pipeline
# =============================================================================

In [ ]:
# ##### Doing a single objective search with weigth objective
# scale_fuel_cost = 40
# scale_risk_cost = 60

# # =============================================================================
# # Graph Construction and Node Creation
# # =============================================================================

# # Instantiate the graph construction object.
# # (Assuming you have already refactored your class into GraphConstructionDiscretization.)
# graph_object = GraphConstructionDiscretization(
#     map_qz, start, goal, q_min, q_max, q_act, alpha, beta, discretization_angle, max_risk_limit, acceptable_risk_limit
# )

# start_time = time.time()
# # Create nodes and the index maps.
# nodes, index_map, reverse_index_map = graph_object.create_nodes()

# # Build the visibility graph
# graph_object.build_visibility_graph(reverse_index_map)
# end_time = time.time()

# print(f"The time req for grap construction: {end_time-start_time}")
# # Assign heuristic cost to each node
# graph_object.assign_heuristic_costs(reverse_index_map)

# def heuristic(n1,n2):
#     return graph_object.visibility_graph.nodes[n1]['heuristic_cost']

# # Run the biobjective search
# start_state = "s"
# goal_state = "g"
# path_cost_fuel_list = []
# path_cost_risk_list = []
# for factor in np.arange(0,1,0.01):
#     for edge_1, edge_2 in graph_object.visibility_graph.edges:
#         graph_object.visibility_graph[edge_1][edge_2]['weight'] = factor*graph_object.visibility_graph[edge_1][edge_2]['fuel_cost'] + (1-factor)*graph_object.visibility_graph[edge_1][edge_2]['risk_cost']

#     path = nx.astar_path(graph_object.visibility_graph, source=start_state, target=goal_state, heuristic=heuristic, weight='weight')
#     print(path)
#     path_cost_fuel = sum([graph_object.visibility_graph[path[i]][path[i+1]]['fuel_cost'] for i in range(len(path)-1)])
#     path_cost_risk = sum([graph_object.visibility_graph[path[i]][path[i+1]]['risk_cost'] for i in range(len(path)-1)])
    
#     path_cost_fuel_list.append(path_cost_fuel)
#     path_cost_risk_list.append(path_cost_risk)
#     print(f"factor: {factor}, path_cost_fuel: {path_cost_fuel}, path_cost_risk: {path_cost_risk}")

# dominated_indices = check_pareto_optimality(path_cost_fuel_list, path_cost_risk_list)

# plt.figure(figsize=(14, 10))
# plt.scatter(fuel_costs, risk_costs, color='royalblue', edgecolors='black',
#             alpha=0.75, s=80)
# plt.scatter(path_cost_fuel_list, path_cost_risk_list, color='red', marker='*')
# plt.xlabel("Fuel Cost", fontsize=14, fontweight='bold')
# plt.ylabel("Risk Cost", fontsize=14, fontweight='bold')
# plt.title("Fuel Cost vs. Risk Cost", fontsize=16, fontweight='bold')
# plt.grid(True, linestyle='--', alpha=0.6)
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# plt.show()